## Decorators

Decorators is a bit of syntactic sugar that has the purpose of reducing duplicate code. It is a nicer way of implementing higher order functions that are specifically designed to add some wrapping behavior to any function.

In [9]:
def greeter(function):
    def wrapper(*args, **kwargs):
        print("Hi!")
        print(function(*args, **kwargs))
        print("Bye!")
    return wrapper

@greeter
def add(x, y):
    return x + y

add("Str", "ing")

Hi!
String
Bye!


In [11]:
from time import time
def timer(f):
    def g(*args, **kwargs):
        start = time()
        ret = f(*args, **kwargs)
        end = time()
        print(end-start)
        return ret
    return g

In [18]:
@timer
def bogosort(lst):
    import random
    while not is_sorted(lst):
        random.shuffle(lst)
    return lst

def is_sorted(a):
    n = len(a)
    for i in range(0, n-1):
        if (a[i] > a[i+1] ):
            return False
    return True

print(bogosort([3,1,2,4,5,9,6,7,34]))

0.05867409706115723
[1, 2, 3, 4, 5, 6, 7, 9, 34]


## Generators

Generators are a way of sequencing things in such a way that priority/control is given back to the caller until needed again.

In [22]:
from time import sleep
def do_stuff():
    ret = []
    for i in range(10):
        sleep(.5)                       # sleep represents some time extensive computation
        ret.append(i)
    return ret

In [23]:
def do_stuff_():
    for i in range(10):
        sleep(.5)
        yield i

In [24]:
for i in do_stuff():
    print(i)

0
1
2
3
4
5
6
7
8
9


In [25]:
for i in do_stuff_():
    print(i)

0
1
2
3
4
5
6
7
8
9


This is annoying design to use.

In [ ]:
class SomeIdiot:
    def do_this_first():
        pass
    def do_this_second():
        pass
    def do_this_last()
        pass

We can instead interleave this code using a generator to sequence them.

In [ ]:
class Smart:
    def do_this():
        _first()
        yield
        _second()
        yield
        _last()
    
    def _first():
        pass
    def _second():
        pass
    def _last()
        pass

## Context Manager

Combining the above ideas together, we get the idea of a context manager which is a way to enforce that some specific setup and teardown actions occur. The common use scenario is as a way of managing resources so we don't run out of memory by not tearing down processes that we are done with.

In [32]:
class File:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.open_file = open(self.filename, self.mode)
        return self.open_file

    def __exit__(self, *args):
        self.open_file.close()
        
with File('stuff.txt', 'w') as f:
    f.write('blah')

Abstracting this process gives us the context manager

In [89]:
class contextmanager:
    def __init__(self, generator):
        self.generator = generator

    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    
    def __enter__(self):
        self.gen_instance = self.generator(*self.args, **self.kwargs)
        return next(self.gen_instance)
    
    def __exit__(self, *args, **kwargs):
        next(self.gen_instance, None)

        
def file(filename, mode):
    f = open(filename, mode)
    yield f
    f.close()

file = contextmanager(file)

with file('stuff.txt', 'w') as f:
    f.write('blah')

Python already has this as a library that we can use

In [55]:
from contextlib import contextmanager

@contextmanager
def file(filename, mode):
    f = open(filename, mode)
    yield f
    f.close()

with file('stuff.txt', 'w') as f:
    f.write('blah')